In [2]:
import pandas as pd
import datetime
import xgboost as xgb
from sklearn.model_selection import train_test_split, TimeSeriesSplit

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
# Any results you write to the current directory are saved as output.

train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
sample = pd.read_csv('../input/sample_submission.csv')

['sample_submission.csv', 'test.csv', 'train.csv']


In [3]:
train.describe()

,store,item,sales
count,913000.000000,913000.000000,913000.000000
mean,5.500000,25.500000,52.250287
std,2.872283,14.430878,28.801144
min,1.000000,1.000000,0.000000
25%,3.000000,13.000000,30.000000
50%,5.500000,25.500000,47.000000
75%,8.000000,38.000000,70.000000
max,10.000000,50.000000,231.000000


In [4]:
train.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [5]:
train.columns

Index(['date', 'store', 'item', 'sales'], dtype='object')

In [6]:
test.columns

Index(['id', 'date', 'store', 'item'], dtype='object')

In [7]:
test.head()

,id,date,store,item
0,0,2018-01-01,1,1
1,1,2018-01-02,1,1
2,2,2018-01-03,1,1
3,3,2018-01-04,1,1
4,4,2018-01-05,1,1


In [8]:
print("train shape:", train.shape)
print("Test shape:", test.shape)
df = pd.concat([train,test])
print(df.shape)
df.head()

train shape: (913000, 4)
Test shape: (45000, 4)
(958000, 5)


C:\Users\Administrator\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,date,id,item,sales,store
0,2013-01-01,NaN,1,13.0,1
1,2013-01-02,NaN,1,11.0,1
2,2013-01-03,NaN,1,14.0,1
3,2013-01-04,NaN,1,13.0,1
4,2013-01-05,NaN,1,10.0,1


#### 日期特点

In [9]:
df['date'] = pd.to_datetime(df['date'],infer_datetime_format=True)


df['month'] = df['date'].dt.month
df['weekday'] = df['date'].dt.dayofweek
df['year'] = df['date'].dt.year
# df['date'].dt.
df['week_of_year']  = df.date.dt.weekofyear
df.head()

,date,id,item,sales,store,month,weekday,year,week_of_year
0,2013-01-01,NaN,1,13.0,1,1,1,2013,1
1,2013-01-02,NaN,1,11.0,1,1,2,2013,1
2,2013-01-03,NaN,1,14.0,1,1,3,2013,1
3,2013-01-04,NaN,1,13.0,1,1,4,2013,1
4,2013-01-05,NaN,1,10.0,1,1,5,2013,1


In [10]:
df.set_index("date",inplace=True) #改变Index
df.head()

,id,item,sales,store,month,weekday,year,week_of_year
date,,,,,,,,
2013-01-01,NaN,1,13.0,1,1,1,2013,1
2013-01-02,NaN,1,11.0,1,1,2,2013,1
2013-01-03,NaN,1,14.0,1,1,3,2013,1
2013-01-04,NaN,1,13.0,1,1,4,2013,1
2013-01-05,NaN,1,10.0,1,1,5,2013,1


In [11]:
df["median-store_item-month"] = df.groupby(['month',"item","store"])["sales"].transform("median")# 该月该商品在该商店的销售量的中值
df["mean-store_item-week"] = df.groupby(['week_of_year',"item","store"])["sales"].transform("mean")# 在一年的这个周内该商品在该商店的平均销售量
df["item-month-sum"] = df.groupby(['month',"item"])["sales"].transform("sum") # 该月该商品在所有商店的总销售量
df["store-month-sum"] = df.groupby(['month',"store"])["sales"].transform("sum") # 该商店这个月所有商品的总销售量
df

,id,item,sales,store,month,weekday,year,week_of_year,median-store_item-month,mean-store_item-week,item-month-sum,store-month-sum
date,,,,,,,,,,,,
2013-01-01,NaN,1,13.0,1,1,1,2013,1,13.0,13.970588,22987.0,249352.0
2013-01-02,NaN,1,11.0,1,1,2,2013,1,13.0,13.970588,22987.0,249352.0
2013-01-03,NaN,1,14.0,1,1,3,2013,1,13.0,13.970588,22987.0,249352.0
2013-01-04,NaN,1,13.0,1,1,4,2013,1,13.0,13.970588,22987.0,249352.0
2013-01-05,NaN,1,10.0,1,1,5,2013,1,13.0,13.970588,22987.0,249352.0
2013-01-06,NaN,1,12.0,1,1,6,2013,1,13.0,13.970588,22987.0,249352.0
2013-01-07,NaN,1,10.0,1,1,0,2013,2,13.0,13.200000,22987.0,249352.0
2013-01-08,NaN,1,9.0,1,1,1,2013,2,13.0,13.200000,22987.0,249352.0
2013-01-09,NaN,1,12.0,1,1,2,2013,2,13.0,13.200000,22987.0,249352.0


In [12]:
# get shifted features for grouped data. Note need to sort first! 
df['store_item_shifted-90'] = df.groupby(["item","store"])['sales'].transform(lambda x:x.shift(90)) # 该商店三个月前的销售量
df['store_item_shifted-180'] = df.groupby(["item","store"])['sales'].transform(lambda x:x.shift(180)) # 该商店六个月前的销售量
df['store_item_shifted-365'] = df.groupby(["item","store"])['sales'].transform(lambda x:x.shift(365)) # 该商店一年前的销售量

df["item-week_shifted-90"] = df.groupby(['week_of_year',"item"])["sales"].transform(lambda x:x.shift(12).sum()) # 每个商品十二周前的总销售量
df["store-week_shifted-90"] = df.groupby(['week_of_year',"store"])["sales"].transform(lambda x:x.shift(12).sum()) # 每个商店十二周前的总销售量
df["item-week_shifted-90"] = df.groupby(['week_of_year',"item"])["sales"].transform(lambda x:x.shift(12).mean()) # 每个商品十二周前的平均日销售量
df["store-week_shifted-90"] = df.groupby(['week_of_year',"store"])["sales"].transform(lambda x:x.shift(12).mean()) # 每个商店十二周前的平均日销售量
df.tail()

,id,item,sales,store,month,weekday,year,week_of_year,median-store_item-month,mean-store_item-week,item-month-sum,store-month-sum,store_item_shifted-90,store_item_shifted-180,store_item_shifted-365,item-week_shifted-90,store-week_shifted-90
date,,,,,,,,,,,,,,,,,
2018-03-27,44995.0,50,NaN,10,3,1,2018,13,67.0,68.771429,92609.0,411975.0,63.0,82.0,66.0,61.417143,54.788571
2018-03-28,44996.0,50,NaN,10,3,2,2018,13,67.0,68.771429,92609.0,411975.0,59.0,90.0,60.0,61.417143,54.788571
2018-03-29,44997.0,50,NaN,10,3,3,2018,13,67.0,68.771429,92609.0,411975.0,74.0,103.0,73.0,61.417143,54.788571
2018-03-30,44998.0,50,NaN,10,3,4,2018,13,67.0,68.771429,92609.0,411975.0,62.0,99.0,68.0,61.417143,54.788571
2018-03-31,44999.0,50,NaN,10,3,5,2018,13,67.0,68.771429,92609.0,411975.0,82.0,71.0,69.0,61.417143,54.788571


In [13]:
col = [i for i in df.columns if i not in ['date','id']]
y = 'sales'
col

['item',
 'sales',
 'store',
 'month',
 'weekday',
 'year',
 'week_of_year',
 'median-store_item-month',
 'mean-store_item-week',
 'item-month-sum',
 'store-month-sum',
 'store_item_shifted-90',
 'store_item_shifted-180',
 'store_item_shifted-365',
 'item-week_shifted-90',
 'store-week_shifted-90']

In [14]:
train.columns

Index(['date', 'store', 'item', 'sales'], dtype='object')

In [15]:
train = df.loc[~df.sales.isna()]
print("new train",train.shape)
test = df.loc[df.sales.isna()]
print("new test",test.shape)

new train (913000, 17)
new test (45000, 17)


In [16]:
y

'sales'

#### 训练集和验证集分离

In [17]:
train_x, train_cv, y, y_cv = train_test_split(train[col],train[y], test_size=0.15, random_state=42)

In [18]:
print(train_x.shape,train_cv.shape,y.shape,y_cv.shape)

(776050, 16) (136950, 16) (776050,) (136950,)


#### 建模

In [19]:
def XGB_regressor(train_X, train_y, test_X, test_y, feature_names=None, seed_val=2017, num_rounds=500):
    param = {}
    param['objective'] = 'reg:linear'
    param['eta'] = 0.1
    param['max_depth'] = 6
    param['silent'] = 1
    param['eval_metric'] = 'mae'
    param['min_child_weight'] = 1
    param['subsample'] = 0.8
    param['colsample_bytree'] = 0.8
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())

    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)
        
    return model    

In [20]:
model = XGB_regressor(train_X = train_x, train_y = y, test_X = train_cv, test_y = y_cv)
y_test = model.predict(xgb.DMatrix(test[col]), ntree_limit = model.best_ntree_limit)

[0]	train-mae:46.5709	test-mae:46.6042
Multiple eval metrics have been passed: 'test-mae' will be used for early stopping.

Will train until test-mae hasn't improved in 20 rounds.
[1]	train-mae:41.9145	test-mae:41.9444
[2]	train-mae:37.7237	test-mae:37.7489
[3]	train-mae:33.9519	test-mae:33.9744
[4]	train-mae:30.5572	test-mae:30.5773
[5]	train-mae:27.502	test-mae:27.5176
[6]	train-mae:24.7524	test-mae:24.7661
[7]	train-mae:22.2778	test-mae:22.2899
[8]	train-mae:20.0506	test-mae:20.0612
[9]	train-mae:18.0466	test-mae:18.0543
[10]	train-mae:16.2422	test-mae:16.2487
[11]	train-mae:14.6185	test-mae:14.624
[12]	train-mae:13.157	test-mae:13.1618
[13]	train-mae:11.8416	test-mae:11.8458
[14]	train-mae:10.6578	test-mae:10.6613
[15]	train-mae:9.59362	test-mae:9.59555
[16]	train-mae:8.63726	test-mae:8.63773
[17]	train-mae:7.77481	test-mae:7.77525
[18]	train-mae:6.99894	test-mae:6.99906
[19]	train-mae:6.30075	test-mae:6.3006
[20]	train-mae:5.67249	test-mae:5.672
[21]	train-mae:5.10747	test-mae:5.1

[193]	train-mae:0.144809	test-mae:0.148037
[194]	train-mae:0.144033	test-mae:0.147248
[195]	train-mae:0.142767	test-mae:0.145996
[196]	train-mae:0.141714	test-mae:0.14494
[197]	train-mae:0.141387	test-mae:0.14462
[198]	train-mae:0.141069	test-mae:0.144312
[199]	train-mae:0.140619	test-mae:0.143864
[200]	train-mae:0.140066	test-mae:0.143331
[201]	train-mae:0.139768	test-mae:0.143039
[202]	train-mae:0.139571	test-mae:0.142851
[203]	train-mae:0.138888	test-mae:0.14216
[204]	train-mae:0.138456	test-mae:0.141728
[205]	train-mae:0.137875	test-mae:0.141142
[206]	train-mae:0.137061	test-mae:0.140304
[207]	train-mae:0.136653	test-mae:0.139883
[208]	train-mae:0.135993	test-mae:0.139212
[209]	train-mae:0.135219	test-mae:0.138424
[210]	train-mae:0.134651	test-mae:0.137831
[211]	train-mae:0.1341	test-mae:0.137282
[212]	train-mae:0.133584	test-mae:0.136757
[213]	train-mae:0.132979	test-mae:0.13613
[214]	train-mae:0.132289	test-mae:0.135448
[215]	train-mae:0.131212	test-mae:0.134355
[216]	train-mae:0

[385]	train-mae:0.094275	test-mae:0.096964
[386]	train-mae:0.094246	test-mae:0.096935
[387]	train-mae:0.094192	test-mae:0.09688
[388]	train-mae:0.094104	test-mae:0.096788
[389]	train-mae:0.094042	test-mae:0.096726
[390]	train-mae:0.093981	test-mae:0.096664
[391]	train-mae:0.093916	test-mae:0.0966
[392]	train-mae:0.093526	test-mae:0.096209
[393]	train-mae:0.093507	test-mae:0.09619
[394]	train-mae:0.093457	test-mae:0.096137
[395]	train-mae:0.093391	test-mae:0.096069
[396]	train-mae:0.093303	test-mae:0.095982
[397]	train-mae:0.093274	test-mae:0.09595
[398]	train-mae:0.093235	test-mae:0.09592
[399]	train-mae:0.09319	test-mae:0.095878
[400]	train-mae:0.093135	test-mae:0.095829
[401]	train-mae:0.09305	test-mae:0.095743
[402]	train-mae:0.092881	test-mae:0.095577
[403]	train-mae:0.092824	test-mae:0.095522
[404]	train-mae:0.092724	test-mae:0.095415
[405]	train-mae:0.092599	test-mae:0.095296
[406]	train-mae:0.09255	test-mae:0.095246
[407]	train-mae:0.092521	test-mae:0.095217
[408]	train-mae:0.09

In [21]:
sample['sales'] = y_test
sample.to_csv('simple_starter.csv', index=False)